# This is a prototype notebook.  Use at your own risk

In [ ]:
%run ../OpenMSI_Arrayed_Analysis_Tools/Arrayed_Analysis_Tools_python_2_or_3.ipy

In [ ]:
#Optional: Load an ArrayedImage from a pickle file. This way you can work off-line
filename="../OpenMSI_Arrayed_Analysis_Tools/demo_spot_save.pkl"
import pickle
img=pickle.load(open(filename,"rb"))
print(img)

In [ ]:
import requests
client = requests.Session()
import getpass
def authenticate_user(client,username):
    password = getpass.getpass()
    authURL = 'https://openmsi.nersc.gov/openmsi/client/login/'
    # Retrieve the CSRF token first
    client.get(authURL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    login_data = dict(username=username, password=password, csrfmiddlewaretoken=csrftoken, next='/')
    r = client.post(authURL, data=login_data, headers=dict(Referer=authURL))
    return client
client = authenticate_user(client,'bpb') # put your username here in the quotes


In [ ]:
payload = {'file':img.filename,
          'expIndex':img.expIndex,'dataIndex':img.dataIndex,'qspectrum_viewerOption':'0',
          'qslice_viewerOption':'0',
          'col':0,'row':0,
          'findPeak':'0','format':'JSON'}
url = 'https://openmsi.nersc.gov/openmsi/qmz'
r = client.get(url,params=payload)
data = json.loads(r.text)
mz = np.asarray(data[u'values_spectra'])

In [ ]:
payload = {'file':img.filename,
              'expIndex':img.expIndex,'dataIndex':img.dataIndex,'qspectrum_viewerOption':'0',
          'qslice_viewerOption':'0','operations':'[{"reduction":"mean","axis":0,"min_dim":2}]',
              'findPeak':'0','format':'JSON'}
url = 'https://openmsi.nersc.gov/openmsi/qspectrum'
spectra_list = []
spotLocations = img.spotLocations
df = {}
df['coords'] = []
df['spectra'] = []
for i,coord in enumerate(img.generateSpotList()):
    payload['col'] = '[' + ','.join([str(c[1]) for c in coord]) + ']'
    payload['row'] = '[' + ','.join([str(c[0]) for c in coord]) + ']'
    r = client.get(url,params=payload)
    data = json.loads(r.text)
    df['coords'].append(coords)
    df['spectra'].append(data)
        